# Calcul des indicateurs (KPIs)

In [1]:
import pandas as pd

# Chargement du fichier Excel
df = pd.read_excel("sales.xlsx")

# Créer les colonnes calculées
df["Sales"] = df["Order Quantity"] * df["Unit Selling Price"]
df["Cost"] = df["Order Quantity"] * df["Unit Cost"]
df["Profit"] = df["Sales"] - df["Cost"]

# Total Sales
total_sales = df["Sales"].sum()

# Total Profit
total_profit = df["Profit"].sum()

# Total Order Quantity
total_orders = df["Order Quantity"].sum()

# Profit Margin %
profit_margin = total_profit / total_sales if total_sales != 0 else 0

# Affichage
print("Indicateurs globaux :")
print(f"Total Sales        : {total_sales:,.2f}")
print(f"Total Profit       : {total_profit:,.2f}")
print(f"Total Orders       : {total_orders:,}")
print(f"Profit Margin (%)  : {profit_margin*100:.2f}%")

#Comparaison avec l’année précédente (YOY – Year Over Year)

df["Sales"] = df["Order Quantity"] * df["Unit Selling Price"]
df["Cost"] = df["Order Quantity"] * df["Unit Cost"]
df["Profit"] = df["Sales"] - df["Cost"]

date_range = pd.date_range(start=df["OrderDate"].min(), end=df["OrderDate"].max(), freq='D')
date_table = pd.DataFrame({"Date": date_range})

date_table["Year"] = date_table["Date"].dt.year
date_table["Month"] = date_table["Date"].dt.month
date_table["Month Name"] = date_table["Date"].dt.strftime('%B')
date_table["Quarter"] = date_table["Date"].dt.quarter
date_table["Day"] = date_table["Date"].dt.day
date_table["Day Name"] = date_table["Date"].dt.strftime('%A')
date_table["Week"] = date_table["Date"].dt.isocalendar().week

date_table_renamed = date_table.rename(columns={"Date": "OrderDate"})
df_merged = pd.merge(df, date_table_renamed, on="OrderDate", how="left")

print("\nComparaison avec l'année précédente :")
# Regrouper par année
sales_by_year = df_merged.groupby("Year").agg({
    "Sales": "sum",
    "Profit": "sum",
    "Order Quantity": "sum"
}).reset_index()

# Renommer pour plus de clarté
sales_by_year.columns = ["Year", "Total Sales", "Total Profit", "Total Order Quantity"]

# Créer les colonnes pour l’année précédente (PY = Previous Year)
sales_by_year["Total Sales PY"] = sales_by_year["Total Sales"].shift(1)
sales_by_year["Sales Var"] = sales_by_year["Total Sales"] - sales_by_year["Total Sales PY"]
sales_by_year["Sales Var %"] = (sales_by_year["Sales Var"] / sales_by_year["Total Sales PY"]) * 100

sales_by_year["Total Profit PY"] = sales_by_year["Total Profit"].shift(1)
sales_by_year["Profit Var"] = sales_by_year["Total Profit"] - sales_by_year["Total Profit PY"]
sales_by_year["Profit Var %"] = (sales_by_year["Profit Var"] / sales_by_year["Total Profit PY"]) * 100

sales_by_year["Order Qty PY"] = sales_by_year["Total Order Quantity"].shift(1)
sales_by_year["Order Qty Var"] = sales_by_year["Total Order Quantity"] - sales_by_year["Order Qty PY"]
sales_by_year["Order Qty Var %"] = (sales_by_year["Order Qty Var"] / sales_by_year["Order Qty PY"]) * 100

# Affichage
print(sales_by_year.round(2))


Indicateurs globaux :
Total Sales        : 154,573,140.60
Total Profit       : 57,789,142.91
Total Orders       : 67,579
Profit Margin (%)  : 37.39%

Comparaison avec l'année précédente :
   Year  Total Sales  Total Profit  Total Order Quantity  Total Sales PY  \
0  2017   52580534.7   19677772.16                 23052             NaN   
1  2018   53463661.7   19789189.55                 23153      52580534.7   
2  2019   48528944.2   18322181.20                 21374      53463661.7   

   Sales Var  Sales Var %  Total Profit PY  Profit Var  Profit Var %  \
0        NaN          NaN              NaN         NaN           NaN   
1   883127.0         1.68      19677772.16   111417.38          0.57   
2 -4934717.5        -9.23      19789189.55 -1467008.35         -7.41   

   Order Qty PY  Order Qty Var  Order Qty Var %  
0           NaN            NaN              NaN  
1       23052.0          101.0             0.44  
2       23153.0        -1779.0            -7.68  
